In [16]:
from patch_and_stitch import PatchStitch

In [17]:
dd = PatchStitch()

In [18]:
import os
import PIL.Image
path = "./post_processed/mse+topo_high_disconn/"
#path = "./prediction/bce_benchmark/2-Final_Results/"
path2 = "./data/example9/MC16_S1/"
#path2 = "./prediction/bce_benchmark/0-Full/Full_raw/img0/"
#imgs = os.listdir(path)
pics = []
#for img in imgs :
#    pics.append((PIL.Image.open(path + img) , PIL.Image.open(path2 + "img0.tif")) )   
#    break

pics.append((PIL.Image.open(path + "MC16_S1.png") , PIL.Image.open(path2 + "img0.tif")))

D:\anaconda\envs\torch_env\lib\site-packages\PIL\Image.py:3167: DecompressionBombWarning: Image size (96657764 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [19]:
pics

[(<PIL.PngImagePlugin.PngImageFile image mode=RGB size=11692x8267>,
  <PIL.TiffImagePlugin.TiffImageFile image mode=RGBA size=11692x8267>)]

In [20]:
pic = pics[0][0]

In [6]:
pic.show()

In [21]:
dd.patch(pic , (5500 , 5500) ,"./final_output/mask" , "pic")

patched


In [22]:
dd1 = PatchStitch()

In [23]:
pic1 = pics[0][1]

In [10]:
pic1.show()

In [24]:
dd1.patch(pic1 , (5500 , 5500) ,"./final_output/image" , "pic")

patched


In [5]:
from grain_identification import *

In [6]:
pic1 = np.array(pic1)
pic = np.array(pic)

In [7]:
pic1 = pic1[:,:,0:3]


In [8]:
pic.shape , pic1.shape

((8267, 11692, 3), (8267, 11692, 3))

In [16]:
pic

array([[[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]]

In [17]:
pic1

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[ 41,  54,  60],
        [ 40,  53,  59],
        [ 40,  53,  59],
        ...,
        [226, 228, 227],
        [226, 228, 227],
        [237, 239, 238]],

       [[ 44,  57,  65],
        [ 42,  55,  61],
        [ 42,  55,  61],
        ...,
        [233, 237, 236],
        [233, 237, 236],
        [234, 236, 235]],

       [[ 44,  57,  65],
        [ 42,  55,  61],
        [ 42,  55,  61],
        ...,
        [233, 237, 236],
        [233, 237, 236],
        [234, 236, 235]]

In [18]:
np.vstack(pic).shape

(96657764, 3)

In [19]:
np.vstack(pic1).shape

(96657764, 3)

In [9]:
arr = np.vstack(pic1)
arr1 = np.vstack(pic)
new_arr = [arr[i] for i in range(len(arr)) if tuple(arr1[i]) != (0 ,0 ,0)]
print(len(arr) , len(arr1) , len(new_arr))

96657764 96657764 76316367


In [10]:
del arr

In [11]:
del arr1

In [12]:
len(new_arr)

76316367

In [13]:
from sklearn.cluster import KMeans
color = [[255 , 0 , 0] , [0,255,0] , [0 , 0 ,255]]
def image_process(k,color , train_data ,loc1 = "./final_output/mask" , loc2 = "./final_output/image" , final_loc = "./final_output/output") :
    maps  = sorted(os.listdir(loc1))
    imgs = sorted(os.listdir(loc2))
    print("Fitting clustering model")
    kmeans = KMeans(n_clusters=k, random_state=0, n_init="auto").fit(train_data)
    del train_data
    for i ,name in enumerate(maps) :
        if i < 3 :
            continue
        print(f"processing {name}")
        map_ = PIL.Image.open(loc1 + '/' + name)
        img_ = PIL.Image.open(loc2 + '/' + name)
        print("getting connected comps")
        dic = get_conn_comps(map_ , img_)
        print("getting vector median")
        map_new = get_vec_median(dic)
        print("getting clusters")
        ans = get_clusters(map_new , k , fit_model = kmeans)
        out_map = get_img_clusters(map_ , ans , dic , colors = color[:k])
        print("saving")
        out_map.save(final_loc + '/' + name)
    
    print("DONE")
    
    return 

In [15]:
image_process(3 , color , new_arr)

Fitting clustering model
processing 5500_0.png
getting connected comps
(2767, 5500)
getting vector median


D:\Proj_file\Code_files\Execution_Scripts\grain_identification.py:55: RuntimeWarning: overflow encountered in scalar subtract
  return math.sqrt((val1[0] - val2[0])**2 + (val1[1] - val2[1])**2 + (val1[2] - val2[2])**2)


getting clusters
saving
processing 5500_11000.png
getting connected comps
(2767, 692)
getting vector median


D:\Proj_file\Code_files\Execution_Scripts\grain_identification.py:55: RuntimeWarning: overflow encountered in scalar subtract
  return math.sqrt((val1[0] - val2[0])**2 + (val1[1] - val2[1])**2 + (val1[2] - val2[2])**2)


getting clusters
saving
processing 5500_5500.png
getting connected comps
(2767, 5500)
getting vector median


D:\Proj_file\Code_files\Execution_Scripts\grain_identification.py:55: RuntimeWarning: overflow encountered in scalar subtract
  return math.sqrt((val1[0] - val2[0])**2 + (val1[1] - val2[1])**2 + (val1[2] - val2[2])**2)


getting clusters
saving
DONE


In [25]:
dd.stitch("./final_output/stitch")

DONE
